In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
conll.keys()

Found cached dataset conll2003 (/Users/kirillsergeev/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 461.05it/s]


dict_keys(['train', 'validation', 'test'])

In [3]:
conll["test"][2]

{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [4]:
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [5]:
from sbe_vallib import NerSampler

In [24]:
sampler = NerSampler(train = {'true': [i for i in conll['train']],
                              'pred': [{'id': i['id'], 'ner_tags': i['ner_tags']} for i in conll['train']]},
                     oos = {'true': [i for i in conll['test']],
                            'pred': [{'id': i['id'], 'ner_tags': i['ner_tags']} for i in conll['test']]})

In [27]:
sampler.set_seed(1, bootstrap=True)
# sampler.oos['pred']

In [28]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [29]:
import typing as tp

start_end_dct = {1:2, 3:4, 5:6, 7:8}


def tokenize_and_preserve_tags(example: tp.Dict[str, tp.Any],
                               tokenizer: BertTokenizer,
                               label2id: tp.Dict[str, int],
                               tokenizer_params={}) -> tp.Dict[str, tp.Any]:
    # write your own function to split each pair of word-token to same number of pieces.

    encoded = tokenizer(example["tokens"], is_split_into_words=True, truncation=True, **tokenizer_params)
    encoded.update(example)

    labels = []
    label = example["ner_tags"]
    # for i, label in enumerate(examples["ner_tags"]):
    #   print(label)
    word_ids = encoded.word_ids(batch_index=0)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        # Special tokens have a word id that is None. We set the label to -100 so they are automatically
        # ignored in the loss function.
        if word_idx is None:
            label_ids.append(0)
        # We set the label for the first token of each word.
        elif word_idx != previous_word_idx:
            label_ids.append(label[word_idx])
        # For the other tokens in a word, we set the label to either the current label or -100, depending on
        # the label_all_tokens flag.
        else:
            # print(label[word_idx])
            if label[word_idx] in start_end_dct:
              label_ids.append(start_end_dct[label[word_idx]])
            else:
              label_ids.append(label[word_idx])
        previous_word_idx = word_idx

    label_ids[0] = 0
    label_ids[-1] = 0
    
    # <YOUR CODE HERE>
    encoded['labels'] = [label2id[CONLL_NER_TAGS[i]] for i in label_ids]
    encoded['text_labels'] = [CONLL_NER_TAGS[i] for i in label_ids]
    
    assert len(encoded['labels']) == len(encoded["input_ids"])
    return encoded
    

In [30]:
test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = tokenize_and_preserve_tags(test_example, tokenizer, model.config.label2id)

In [31]:
test_result

{'input_ids': [101, 1230, 1271, 1110, 5466, 7752, 2142, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'tokens': ['His', 'name', 'is', 'Jerry', 'Abrahamson'], 'ner_tags': [0, 0, 0, 1, 2], 'labels': [0, 0, 0, 0, 3, 4, 4, 0], 'text_labels': ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']}

In [36]:
test_result['labels']

[0, 0, 0, 0, 3, 4, 4, 0]

In [38]:
conll_prepared = conll.map(lambda x: tokenize_and_preserve_tags(x, tokenizer, model.config.label2id))

100%|██████████| 14041/14041 [00:04<00:00, 3205.01ex/s]
Loading cached processed dataset at /Users/kirillsergeev/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-66f873fce4b35a3d.arrow
Loading cached processed dataset at /Users/kirillsergeev/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-eb6b92e96444ffe1.arrow
